In [1]:
import os
import sys
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import MDAnalysis as mda
from MDAnalysis.analysis import distances
from MDAnalysis.analysis import rms
from tqdm import tqdm
from numpy import linalg as LA
from scipy.signal import lfilter

sys.path.append("../py_modules/")

from utils import sparse_to_full, triu_to_full

In [2]:
def dist_pbc(a, b, box):
    """
    calculate distance between two points
    in PBC box
    """
    assert len(a) == len(b)
    box = box[:len(a)]
    a = a % box
    b = b % box
    dist_vec = np.abs(a - b)
    dist_vec = np.abs(dist_vec - box * (dist_vec > box/2))
    return LA.norm(dist_vec)
    print(dist_vec)

In [3]:
runs = sorted(glob.glob("../../traj_save/nsp10_16_dist/*.pdb"))

comp = '../../traj_save/Nsp10_Nsp16/comp_0.pdb'
runs = [comp] + runs
ref_pdb = '../../md_setups_a99/input_comp_sam/comp_sam.pdb'

In [4]:
df = []

for run in runs[:]:
    pdb_file = run
    dcd_file = run.replace("pdb", "dcd")
    run_base = os.path.basename(run)[:-4]

    mda_traj = mda.Universe(pdb_file, dcd_file)
    print(len(mda_traj.trajectory))

    nsp16 = mda_traj.segments[0].atoms
    nsp10 = mda_traj.select_atoms("segid B")
    no_sol = mda_traj.select_atoms("protein")
    
    nsp16_ca = nsp16.select_atoms('name CA') 
    nsp10_ca = nsp10.select_atoms('name CA')
    protein_ca = mda_traj.select_atoms('protein and name CA')
    
    ref_traj = mda.Universe(ref_pdb)
    ref_nsp16 = ref_traj.segments[0].atoms
    ref_nsp10 = ref_traj.select_atoms("segid B")
    ref_nsp16_ca = ref_nsp16.select_atoms('name CA') 
    ref_nsp10_ca = ref_nsp10.select_atoms('name CA')
    ref_protein_ca = ref_traj.select_atoms('protein and name CA')
    
    for ts in tqdm(mda_traj.trajectory[:4000]):
        cm  = (distances.distance_array(
            nsp10_ca.positions, nsp16_ca.positions, 
            box = ts.dimensions) < 10.0) * 1
        dist = dist_pbc(
            nsp10.center_of_mass(), 
            nsp16.center_of_mass(), 
            ts.dimensions,
        )
        rmsd = rms.rmsd(
            protein_ca.positions, ref_protein_ca.positions, 
            superposition=True,
        )
        rmsd_nsp10 = rms.rmsd(
            nsp10_ca.positions, ref_nsp10_ca.positions, 
            superposition=True,
        )
        rmsd_nsp16 = rms.rmsd(
            nsp16_ca.positions, ref_nsp16_ca.positions, 
            superposition=True,
        )
        
        df.append({'sys_name': run_base,
                    'frame': ts.frame,
                   'n_res': protein_ca.n_atoms, 
                   'dist': dist,
                   'n_contacts': np.sum(cm==1),
                   'rmsd': rmsd, 
                   'rmsd_nsp10': rmsd_nsp10, 
                   'rmsd_nsp16': rmsd_nsp16
                   
    #                    'dist_vec': dist_vec,
#                    'contact_maps': cm
                  })

/homes/heng.ma/miniconda3/envs/MD_ff/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:334: UserWarning: Unknown element Z found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/homes/heng.ma/miniconda3/envs/MD_ff/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: Z
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


10000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:05<00:00, 742.66it/s]


4000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:05<00:00, 742.22it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:05<00:00, 734.92it/s]


4000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:05<00:00, 731.34it/s]


4000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:05<00:00, 738.88it/s]


In [5]:
df = pd.DataFrame(df)
df

,sys_name,frame,n_res,dist,n_contacts,rmsd,rmsd_nsp10,rmsd_nsp16
0,comp_0,0,415,32.605477,143,1.270958,1.157980,1.061589
1,comp_0,1,415,32.323373,144,1.266247,1.135788,1.180746
2,comp_0,2,415,31.932924,156,1.251338,1.298619,1.130007
3,comp_0,3,415,32.521470,149,1.232570,1.067588,1.155031
4,comp_0,4,415,32.193099,157,1.497461,1.652155,1.348938
...,...,...,...,...,...,...,...,...
83995,comp_100,3995,415,39.647390,107,10.644501,5.009146,2.698992
83996,comp_100,3996,415,39.943104,114,10.607467,4.521076,2.744293
83997,comp_100,3997,415,39.952250,113,10.840326,4.824111,2.710605
83998,comp_100,3998,415,40.436388,108,10.780112,4.630009,2.768572


In [6]:
df.to_pickle("df_comp.pkl")